# This Program is evaluate the Testing Accuracy On Our Custome created Dataset

In [8]:
# import torch
print(torch.__version__)

1.0.1.post2


In [9]:
 # For custom datasets and this function is used to lode Testing  dataset

from torch.utils.data.dataset import Dataset 
import numpy as np
from PIL import Image
import PIL.ImageOps
from random import *
from random import randint
class CustomDatasetFromImagesForTesting(Dataset):
    def __init__(self, csv_path):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform_def: pytorch transforms for transforms and tensor conversion 
        """
        self.to_pure_tensor =transforms.Compose([
           
            transforms.ToTensor()
            ])
        # Read the csv file
        self.data_info = pd.read_csv(csv_path, header=None)
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        # Third column is for an operation indicator
        self.operation_arr = np.asarray(self.data_info.iloc[:, 2])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = Image.open(single_image_name,'r')
        img_as_img = PIL.ImageOps.invert(img_as_img)
        #convart the image into RGB format
        img_as_img=img_as_img.convert('RGB')
        new_width  = 32
        new_height = 32
        #resizeing the image into 32x32 
        img_as_img = img_as_img.resize((new_width, new_height), Image.ANTIALIAS)
        #adjust the contrast with factor 1.7
        img_as_img=transforms.functional.adjust_contrast(img_as_img,1.7)
        # Check if there is an operation
        some_operation = self.operation_arr[index]
        # If there is an operation
        if some_operation:
            # Do some operation on image
            # ...no operations is happen
            # ...
            pass
        # Transform image to tensor
        img_as_tensor = self.to_pure_tensor(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len
      
      

# Lodeing BDNet own test dataset

In [10]:
# Extracting our custome created datasamples for testing 
from zipfile import ZipFile
zf = ZipFile('own.zip', 'r')
zf.extractall('./')
zf.close()

In [11]:
import argparse
import os
import shutil
import time
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.onnx as ax



import os
import csv
from tqdm import tqdm
import torch,torchvision
import shutil
from PIL import Image
from torch.autograd import Variable





# lode the Best model's state , that is parfect fitted model from above traning process 
the_model = torch.load('./BDNet.pth', map_location='cpu' )   # Since we test on our own pc so, map_location='cpu'
#create an array of 10x10 dimension and initialize each field with 0
c_mat= [[0 for i in range(10)] for i in range(10)]
for kkk in range(10):
  for hhh in range(10):
    c_mat[kkk][hhh]=0
#lode the IsI test datasamples
cvs_file_fr_test_beng =CustomDatasetFromImagesForTesting('./own/owndta.csv')

tesst_loader = torch.utils.data.DataLoader(dataset=cvs_file_fr_test_beng,batch_size=1, shuffle=True)
ttl=0
crect=0
the_model.eval()
for i, (input, target) in enumerate(tesst_loader):
  
  input_var = torch.autograd.Variable(input, volatile=True)
  output = the_model(input_var)
  res=output.max(1, keepdim=True)[1]
  # fff holdes the predicted output class by our model
  # target holdes the actual output class 
  fff=res.tolist()[0][0]
  if(int(target)==int(fff)) :     #if the model predict correctly
     crect = crect+1
  ttl=ttl+1
  c_mat[int(target)][int(fff)]=c_mat[int(target)][int(fff)] + 1
  
print("\n\n Confution matrics on Custome made Our own DataSamples:-\n")
for kkk in range(10):
  for hhh in range(10):
    print(c_mat[kkk][hhh],end="\t")
  print("\n")   
print("Final testing accuracy on Our Own Datasamples: ",(crect/ttl)*100," %")


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.




 Confution matrics on Custome made Our own DataSamples:-

100	0	0	0	0	0	0	0	0	0	

0	100	0	0	0	0	0	0	0	0	

0	0	100	0	0	0	0	0	0	0	

0	0	0	100	0	0	0	0	0	0	

0	0	0	0	100	0	0	0	0	0	

6	0	0	1	0	90	2	0	1	0	

0	0	0	1	0	0	99	0	0	0	

1	0	2	0	0	0	0	97	0	0	

0	0	8	0	0	0	1	0	91	0	

0	2	0	0	0	0	0	0	0	98	

Final testing accuracy on Our Own Datasamples:  97.5  %
